# Anomaly Detection on the Virgin River with FF

Anomaly Detection is a .



In [ ]:
from google.colab import auth
from datetime import datetime
import os
import pandas as pd
auth.authenticate_user()
!git clone https://github.com/AIStream-Peelout/flow-forecast.git
!gsutil cp gs://predict_cfs/virgin_final1.csv virgin_final.csv 
!gsutil cp -r gs://predict_cfs/joined_final_3 .
!gsutil cp gs://aistream-datasets/flowdb/09405500AZC_flow.csv .
df = pd.read_csv("09405500AZC_flow.csv")
df["anomalous_rain"] = df["p01m"].apply(lambda x: 0 if x<1 else 1)
df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("/content/joined_final_3/09405500AZC_flow.csv")

Cloning into 'flow-forecast'...
remote: Enumerating objects: 18191, done.
remote: Counting objects: 100% (604/604), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 18191 (delta 367), reused 484 (delta 313), pack-reused 17587
Receiving objects: 100% (18191/18191), 4.72 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (13072/13072), done.
Copying gs://predict_cfs/virgin_final1.csv...
-
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://predict_cfs/joined_final_3/0101000040B_flow.csv...
Copying gs://predict_cfs/joined_final_3/0101007040B_flow.csv...
Copying gs://predict_cfs/joined_final_3/01010500FVE_flow.csv...
Copying gs://predict_cfs/joined_final_3/0101100040B_flow.csv...
- [4 files][ 98.5 MiB/ 98.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil h

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,7,8,9,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
the_config = {                 
   "model_name": "CustomTransformerDecoder",
   "n_targets": 2,
   "model_type": "PyTorch",
    "model_params": {
      "n_time_series":4,
      "seq_length":26,
      "output_seq_length": 1, 
      "output_dim":2,
      "n_layers_encoder": 6
     }, 
    "dataset_params":
    { "class": "GeneralClassificationLoader",
      "n_classes": 2,
       "training_path": "/content/joined_final_3/09405500AZC_flow.csv",
       "validation_path": "/content/joined_final_3/09405500AZC_flow.csv",
       "test_path": "/content/joined_final_3/09405500AZC_flow.csv",
       "sequence_length":26,
       "batch_size":4,
       "forecast_history":26,
       "train_end": 4500,
       "valid_start":4501,
       "valid_end": 7000,
      
       "target_col": ["anomalous_rain"],
       "relevant_cols": ["anomalous_rain"] + ["tmpf", "cfs", "dwpf", "height"],
       "scaler": "StandardScaler", 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]
           }

       },
    },

    "training_params":
    {
       "criterion":"CrossEntropyLoss",
       "optimizer": "Adam",
       "optim_params":
       {},
       "lr": 0.03,
       "epochs": 4,
       "batch_size":100
    },
    "GCS": True,
   
    "wandb": {
       "name": "flood_forecast_anomaly_virign",
       "tags": ["anomaly_detection", "virgin_river", "simple", "classification"],
       "project": "repo-flood_forecast"
    },
   "forward_params":{},
   "metrics":["CrossEntropyLoss"]
}

In [ ]:
df["anomalous_rain"] = df["p01m"].apply(lambda x: 0 if x<1 else 1)


In [ ]:
#from flood_forecast.trainer import train_function
os.chdir("flow-forecast")
!pip install -r  requirements.txt
!python setup.py develop
!wandb login
os.environ['MODEL_BUCKET'] = "predict_cfs"
os.environ["ENVIRONMENT_GCP"] = "Colab"
os.environ["GCP_PROJECT"] = "gmap-997"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 5.3 MB/s 
     |████████████████████████████████| 6.0 MB 39.6 MB/s 
     |████████████████████████████████| 1.8 MB 60.5 MB/s 
     |████████████████████████████████| 28.8 MB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 51.4 MB/s 
     |████████████████████████████████| 201 kB 45.6 MB/s 
     |████████████████████████████████| 2.8 MB 34.5 MB/s 
     |████████████████████████████████| 182 kB 66.9 MB/s 
     |████████████████████████████████| 166 kB 56.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 166 kB 65.3 MB/s 
     |████████████████████████████████| 162 kB 58.1 MB/s 
     |████████████████████████████████| 162 kB 50.9 MB/s 
     |████████████████████████████████| 158 kB 62.0 MB/s 
     |████████████████████████████████| 157 kB 69.6 MB/s 
     |████████████████████████████

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:530: UserWarning: Normalizing '0.999998dev' to '0.999998.dev0'
  warnings.warn(tmpl.format(**locals()))
running develop
/usr/local/lib/python3.7/dist-packages/setuptools/command/easy_install.py:147: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
/usr/local/lib/python3.7/dist-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt
writing top-level names to flood_forecast.egg-info/top_level.txt
writing manifest file 'flood_forecast.egg-info/SOURCES.txt'
adding licens

In [ ]:
df.to_csv("/content/joined_final_3/09405500AZC_flow.csv")
from flood_forecast.trainer import train_function

In [ ]:
trained_model = train_function("PyTorch", the_config)

In [ ]:
import flood_forecast
from flood_forecast.deployment.inference import InferenceMode

In [ ]:
import json
with open("/content/flow-forecast/model_save/30_May_202206_53PM.json") as f:
  data = json.load(f)

In [ ]:
data["inference_params"] = {}
data["inference_params"]["dataset_params"] = data["dataset_params"]
m = InferenceMode(1, 1, data, "/content/joined_final_3/09405500AZC_flow.csv", "/content/flow-forecast/model_save/24_May_202202_25PM_model.pth")

In [ ]:
from flood_forecast.preprocessing.pytorch_loaders import GeneralClassificationLoader
def infer_now_classification(model, data=None, over_lap_seq=True, save_buck=None, save_name=None, use_torch_script=False):
    """Function to preform classification/anomaly detection on sequences in real-time
    """
    if data:
        dataset_params = model.model.params["dataset_params"].copy()
        dataset_params["class"] = "GeneralClassificationLoader"
        loader = m.model.make_data_load(data, dataset_params, "custom") 
    else:
        loader = model.model.test_data
    seq_list = []
    if over_lap_seq:
        for x, y in loader:
            seq_list.append(model.model.model(x))
    return seq_list

In [ ]:
import pandas as pd
pd.read_csv("/content/joined_final_3/09405500AZC_flow.csv")[0:200].to_csv("test.csv")

In [ ]:
res = m.infer_now_classification("test.csv")

In [ ]:
import torch
d = torch.nn.Softmax(dim=1)
d1 = map( lambda x: d(x[0]), res)
list(d1)

In [ ]:
df[1:90].to_csv("ff1_tes.csv")

In [ ]:
!gsutil cp gs://flow_datasets/test_data/model_save/24_May_202202_25PM_model.pth model_save 

In [ ]:
!cp model_save/test_data/model_save/23_May_202203_31PM_model.pth /content/flow-forecast/model_save/test_data/23_May_202203_31PM_model.pth